In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
# matplotlib 한글 꺠짐 방지
# apt-get update
# apt-get install fonts-nanum* 
# apt-get install fontconfig
# fc-cache -fv  # font 캐시 날리기
# rm -rf /home/hy1/.cache/matplotlib/  #matplotliob 폰트 캐시 날리기



In [2]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import *
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 


In [3]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [4]:
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    rc('font', family='NanumGothic')

## Load - DataWareHouse

## JDBC DICT

In [12]:
JDBC = {
    'url':'jdbc:oracle:thin:@decorona_high?TNS_ADMIN=/home/big/study/db/Wallet_DECORONA',
    'props':{
        'user':'dw_admin',
        'password':'123qwe!@#QWE'
    }
}

### 1. LOC 테이블 저장

In [7]:
area = spark.read.csv('/corona_data/loc/sido_area.csv', encoding='CP949', header=True)
popu = spark.read.csv('/corona_data/loc/sido_population.csv', encoding='CP949', header=True)


In [7]:
# loc 데이터 저장
area_pop = area.join(popu, on='loc')
area_pop = area_pop.select(col('loc').alias('LOC')
                            ,col('area').alias('AREA')
                            ,col('total').alias('POPULATION'))
area_pop.write.jdbc(url=JDBC['url'], table='LOC', mode='append', properties=JDBC['props'])

### 2. CORONA_PATIENT 테이블 저장

In [10]:
# 코로나 감염자 데이터
path = '/corona_data/patient'
co_patient_json = spark.read.json(path, encoding='UTF-8')
co_patient_json.show()

+----------------------+---------+------+----------+--------------+----------+
|                 items|numOfRows|pageNo|resultCode|     resultMsg|totalCount|
+----------------------+---------+------+----------+--------------+----------+
|[{16, 12659, 검역, ...|      500|     1|        00|NORMAL SERVICE|     19684|
|  [{1465, 1461477, ...|      500|     1|        00|NORMAL SERVICE|     19684|
|[{16, 12440, 검역, ...|      500|     1|        00|NORMAL SERVICE|     19684|
| [{1043, 868166, 충...|      500|     1|        00|NORMAL SERVICE|     19684|
|  [{1429, 1214957, ...|      500|     1|        00|NORMAL SERVICE|     19684|
|[{528, 687990, 전남...|      500|     1|        00|NORMAL SERVICE|     19684|
|  [{26161, 22449475...|      500|     1|        00|NORMAL SERVICE|     19684|
|  [{25441, 20983169...|      500|     1|        00|NORMAL SERVICE|     19684|
| [{1376, 847452, 대...|      500|     1|        00|NORMAL SERVICE|     19684|
|[{520, 666170, 전남...|      500|     1|        00|NORMAL SER

In [15]:
data = []
for r1 in co_patient_json.select('items').toLocalIterator():
    if not r1.items:
        continue
    for r2 in r1.items:
        data.append(r2)

patient_data = spark.createDataFrame(data)
patient_data.show(3)

+--------+-------+-----+-------+-----------+------+------------+----------+-----------+-----------+-------+----------+
|deathCnt| defCnt|gubun|gubunCn|    gubunEn|incDec|isolClearCnt|isolIngCnt|localOccCnt|overFlowCnt|qurRate|    stdDay|
+--------+-------+-----+-------+-----------+------+------------+----------+-----------+-----------+-------+----------+
|      16|  12659| 검역| 隔離區|  Lazaretto|    17|           0|         0|          0|         17|      -|2022-08-25|
|      16|  12659| 검역| 隔離區|  Lazaretto|    17|           0|         0|          0|         17|      -|2022-08-25|
|    6715|6109867| 경기|   京畿|Gyeonggi-do| 27032|           0|         0|      27007|         25|  45040|2022-08-25|
+--------+-------+-----+-------+-----------+------+------------+----------+-----------+-----------+-------+----------+
only showing top 3 rows



In [16]:
co_patients = patient_data.select(
    patient_data.gubun.alias('LOC')
    ,patient_data.deathCnt.alias('DEATH_CNT')
     ,patient_data.defCnt.alias('DEF_CNT')
     ,patient_data.localOccCnt.alias('LOC_OCC_CNT')
     ,patient_data.qurRate.alias('QUR_RATE')
     ,patient_data.stdDay.alias('STD_DAY')
).where(~(col('LOC').isin(['합계','검역'])) ).distinct()
co_patients.printSchema()

root
 |-- LOC: string (nullable = true)
 |-- DEATH_CNT: string (nullable = true)
 |-- DEF_CNT: string (nullable = true)
 |-- LOC_OCC_CNT: string (nullable = true)
 |-- QUR_RATE: string (nullable = true)
 |-- STD_DAY: string (nullable = true)



In [17]:
co_patients.write.jdbc(url=JDBC['url'], table='CORONA_PATIENTS', mode='append', properties=JDBC['props'])

### 3.  CORONA_VACCINE 테이블 저장

In [5]:
file_name = '/corona_data/vaccine/'
vaccine = spark.read.json(file_name, multiLine=True)

# vaccine.select(vaccine.data).show()
# vaccine.printSchema()
vaccine.show()


+----------------------+-----------------------------+
|                  data|                         meta|
+----------------------+-----------------------------+
|[{서울, 8340502, 82...|{{지역, 1차접종자수, 2차접...|
+----------------------+-----------------------------+



In [6]:
vaccine.select('meta').collect()

[Row(meta=Row(cols=Row(loc='지역', v1='1차접종자수', v2='2차접종자수', v3='3차접종자수', v4='4차접종자수'), desc='지역별 코로나 예방접종 인구 현황', std_day='2022-09-15'))]

In [13]:
data = []
for r1 in vaccine.select(vaccine.data, vaccine.meta.std_day).toLocalIterator():
    for r2 in r1.data:
        # row 객체를 만들기 위해 dict 생성
        # ** 파이썬 압축해제 키워드 
        # **kwargs   => 여러 쌍의 키워드 args 가 넘어오면 받아서 dictionary로 반환
        # fnc(**dict) => dict에 있는 key-value 값들이 여러쌍의 kwargs  형태로 함수에 전달
        temp = r2.asDict()
        temp['std_day'] = r1['meta.std_day']
        data.append(Row(**temp))
        
vaccine_data = spark.createDataFrame(data)
vaccine_data.show(3)

    

+----+-------+-------+-------+-------+----------+
| loc|     v1|     v2|     v3|     v4|   std_day|
+----+-------+-------+-------+-------+----------+
|서울|8340502|8261480|6057684|1226070|2022-09-15|
|부산|2879898|2850966|2134635| 488630|2022-09-15|
|대구|2019295|1996343|1402649| 259429|2022-09-15|
+----+-------+-------+-------+-------+----------+
only showing top 3 rows



In [30]:
# column을 row로 : stack
# row를 column : pivot

# sql의 dataframe -> pandas의 dataframe
pd_vaccine = vaccine_data.to_pandas_on_spark() 
pd_vaccine = pd_vaccine.set_index(['loc','std_day']) # 인덱스로 설정한 컬럼은 stack되지 않음
pd_vaccine = pd_vaccine.stack() # 현재 series
pd_vaccine = pd_vaccine.to_dataframe('V_CNT')# 다시 df로 변환
pd_vaccine

V_CNT
loc std_day                
서울  2022-09-15 v2   8261480
               v4   1226070
               v1   8340502
               v3   6057684
부산  2022-09-15 v2   2850966
               v4    488630
               v1   2879898
               v3   2134635
대구  2022-09-15 v2   1996343
               v4    259429
               v1   2019295
               v3   1402649
인천  2022-09-15 v2   2546081
               v4    394343
               v1   2570641
               v3   1907586
광주  2022-09-15 v2   1251019
               v4    233610
               v1   1262690
               v3    968067
대전  2022-09-15 v2   1233946
               v4    193191
               v1   1246228
               v3    907587
울산  2022-09-15 v2    959237
               v4    124886
               v1    968595
               v3    719491
세종  2022-09-15 v2    294823
               v4     34585
               v1    298593
               v3    209151
경기  2022-09-15 v2  11711865
               v4   1680362
               v1  11827822
               v3   8677235
강원  2022-09-15 v2   1338931
               v4    269222
               v1   1350315
               v3   1062086
충북  2022-09-15 v2   1416910
               v4    277332
               v1   1429582
               v3   1105875
충남  2022-09-15 v2   1881779
               v4    365655
               v1   1899475
               v3   1475491
전북  2022-09-15 v2   1580541
               v4    370187
               v1   1593746
               v3   1284133
전남  2022-09-15 v2   1640068
               v4    444061
               v1   1654751
               v3   1363212
경북  2022-09-15 v2   2269405
               v4    394769
               v1   2293357
               v3   1724517
경남  2022-09-15 v2   2854106
               v4    483703
               v1   2883515
               v3   2142110
제주  2022-09-15 v2    584814
               v4     92009
               v1    590581
               v3    437805

In [33]:
# 멀티 인덱스를 다시 뜯어줌
# print(pd_vaccine.index)
pd_vaccine = pd_vaccine.reset_index() # 숫자 인덱스로 변환
# print(pd_vaccine.index)

# spark모양 판다스로 다시 변환
pd_vaccine = pd_vaccine.rename(columns={'level_2':'V_TH'})
vaccine_data = pd_vaccine.to_spark()
vaccine_data.show(3)

+-------+-----+----+----------+----+-------+
|level_0|index| loc|   std_day|V_TH|  V_CNT|
+-------+-----+----+----------+----+-------+
|      0|    0|서울|2022-09-15|  v2|8261480|
|      1|    1|서울|2022-09-15|  v4|1226070|
|      2|    2|서울|2022-09-15|  v1|8340502|
+-------+-----+----+----------+----+-------+
only showing top 3 rows



In [37]:
vaccine_data = vaccine_data.drop('level_0')
vaccine_data = vaccine_data.drop('index')
vaccine_data.show(3)

+----+----------+----+-------+
| loc|   std_day|V_TH|  V_CNT|
+----+----------+----+-------+
|서울|2022-09-15|  v2|8261480|
|서울|2022-09-15|  v4|1226070|
|서울|2022-09-15|  v1|8340502|
+----+----------+----+-------+
only showing top 3 rows



In [38]:
vaccine_data.write.jdbc(url=JDBC['url'], table='CORONA_VACCINE', mode='append', properties=JDBC['props'])

### 4. LOC_FACILITY_CNT	테이블 저장